In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
limp = lazy_imports()

import pandas as pd
import numpy as np
from dask import dataframe as dd
from pyspark import SparkContext
from openpyxl import load_workbook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import dash
import bokeh
import altair as alt
import pydot
import statistics
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
import keras
import nltk
import gensim
import spacy
import re
import sys
import os
import glob
from pathlib import Path
import pickle
import datetime as 

In [99]:
# Read in training data
train_data = Path("data/train.csv")
train_df = pd.read_csv(train_data, delimiter=",", index_col='Id')
train_df = train_df.fillna('0')
train_df = pd.get_dummies(train_df)
train_df.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,8450,7,5,2003,2003,706,0,150,856,...,0,0,0,1,0,0,0,0,1,0
2,20,9600,6,8,1976,1976,978,0,284,1262,...,0,0,0,1,0,0,0,0,1,0
3,60,11250,7,5,2001,2002,486,0,434,920,...,0,0,0,1,0,0,0,0,1,0
4,70,9550,7,5,1915,1970,216,0,540,756,...,0,0,0,1,1,0,0,0,0,0
5,60,14260,8,5,2000,2000,655,0,490,1145,...,0,0,0,1,0,0,0,0,1,0


In [100]:
# Create the features (X) and target (y) sets
X = train_df.drop(['SalePrice'], axis=1)
y = train_df["SalePrice"]

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [102]:
# Creating StandardScaler instance
scaler = sklearn.preprocessing.StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [103]:
max_feat = round(len(train_df.columns)*.75)

In [104]:
max_feat

629

In [105]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    regressor = GradientBoostingRegressor(n_estimators=200,
                                            learning_rate=learning_rate,
                                            max_features=max_feat,
                                            max_depth=5,
                                            random_state=0)

    # Fit the model
    regressor.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.5f}".format(
        regressor.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (test): {0:.5f}".format(
        regressor.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.99257
Accuracy score (test): 0.85614

Learning rate:  0.1
Accuracy score (training): 0.99582
Accuracy score (test): 0.86830

Learning rate:  0.25
Accuracy score (training): 0.99893
Accuracy score (test): 0.85257

Learning rate:  0.5
Accuracy score (training): 0.99988
Accuracy score (test): 0.81130

Learning rate:  0.75
Accuracy score (training): 0.99999
Accuracy score (test): 0.83660

Learning rate:  1
Accuracy score (training): 1.00000
Accuracy score (test): 0.78011



In [106]:
# Choose a learning rate and create classifier
regressor = GradientBoostingRegressor(n_estimators=200,
                                        learning_rate=0.25,
                                        max_features=5,
                                        max_depth=5,
                                        random_state=0)

# Fit the model
regressor.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = regressor.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).tail(20)

,Prediction,Actual
345,256377.495172,274000
346,114257.762523,154500
347,196862.501612,184000
348,260992.388692,225000
349,121377.038117,130000
350,121364.210034,115000
351,240858.994796,228000
352,197760.270689,265900
353,213817.626275,207500
354,220902.040422,225000


In [108]:
test_data = Path("data/test.csv")
test_df = pd.read_csv(test_data, delimiter=",", index_col='Id')
test_df = test_df.fillna('0')
test_df = pd.get_dummies(test_df)
test_df.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,11622,5,6,1961,1961,896,0,0,896,...,0,0,0,1,0,0,0,0,1,0
1462,20,14267,6,6,1958,1958,1329,0,0,1329,...,0,0,0,1,0,0,0,0,1,0
1463,60,13830,5,5,1997,1998,928,701,0,1629,...,0,0,0,1,0,0,0,0,1,0
1464,60,9978,6,6,1998,1998,926,678,0,1604,...,0,0,0,1,0,0,0,0,1,0
1465,120,5005,8,5,1992,1992,1280,0,0,1280,...,0,0,0,1,0,0,0,0,1,0


In [109]:
test_scaled = X_scaler.transform(test_df)

ValueError: operands could not be broadcast together with shapes (1459,3638) (838,) (1459,3638) 

In [110]:
# Make Prediction
final_predictions = regressor.predict(test_scaled)
final = pd.DataFrame({"id":test_df.index,"SalePrice": final_predictions})

ValueError: arrays must all be same length

In [49]:
final

,id,SalePrice
0,1,201864.783706
1,2,170992.542169
2,3,225698.991080
3,4,151524.271580
4,5,262537.218572
...,...,...
1455,1456,179119.165692
1456,1457,200368.578410
1457,1458,273094.648453
1458,1459,142004.398858


In [53]:
final.to_csv('submission.csv', index= False)

In [92]:
X_train

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
382,20,7200,7,5,2006,2006,0,0,1293,1293,...,0,1,0,0,0,0,0,0,0,1
709,60,9018,7,5,2007,2007,0,0,728,728,...,0,1,0,0,0,0,0,0,0,1
480,30,5925,4,7,1937,2000,168,0,739,907,...,0,0,0,1,0,0,1,0,0,0
809,80,13400,5,5,1966,1966,516,128,380,1024,...,0,0,0,1,0,0,0,0,1,0
1189,60,8935,7,5,2002,2002,0,0,831,831,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,50,12665,5,8,1915,1950,0,0,876,876,...,0,0,0,1,0,0,0,0,1,0
1218,20,8640,8,5,2009,2009,936,0,364,1300,...,0,1,0,0,0,0,0,0,0,1
1193,50,9600,5,8,1925,1994,0,0,702,702,...,0,0,0,1,0,0,0,0,1,0


In [93]:
train_df

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,8450,7,5,2003,2003,706,0,150,856,...,0,0,0,1,0,0,0,0,1,0
2,20,9600,6,8,1976,1976,978,0,284,1262,...,0,0,0,1,0,0,0,0,1,0
3,60,11250,7,5,2001,2002,486,0,434,920,...,0,0,0,1,0,0,0,0,1,0
4,70,9550,7,5,1915,1970,216,0,540,756,...,0,0,0,1,1,0,0,0,0,0
5,60,14260,8,5,2000,2000,655,0,490,1145,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,7917,6,5,1999,2000,0,0,953,953,...,0,0,0,1,0,0,0,0,1,0
1457,20,13175,6,6,1978,1988,790,163,589,1542,...,0,0,0,1,0,0,0,0,1,0
1458,70,9042,7,9,1941,2006,275,0,877,1152,...,0,0,0,1,0,0,0,0,1,0


In [90]:
train_not_test = train_cols.difference(test_cols)

In [89]:
train_cols = train_df.columns
test_cols = test_df.columns

In [91]:
train_not_test

Index(['BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'Electrical_0',
       'Electrical_Mix',
       ...
       'MasVnrArea_975.0', 'MiscFeature_TenC', 'PoolQC_Fa', 'RoofMatl_ClyTile',
       'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'SalePrice',
       'TotalBsmtSF', 'Utilities_NoSeWa'],
      dtype='object', length=188)

In [94]:
test_df

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,11622,5,6,1961,1961,896,0,0,896,...,0,0,0,1,0,0,0,0,1,0
1462,20,14267,6,6,1958,1958,1329,0,0,1329,...,0,0,0,1,0,0,0,0,1,0
1463,60,13830,5,5,1997,1998,928,701,0,1629,...,0,0,0,1,0,0,0,0,1,0
1464,60,9978,6,6,1998,1998,926,678,0,1604,...,0,0,0,1,0,0,0,0,1,0
1465,120,5005,8,5,1992,1992,1280,0,0,1280,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,1936,4,7,1970,1970,546,546,0,1092,...,0,0,0,1,0,0,0,0,1,0
2916,160,1894,4,5,1970,1970,546,546,0,1092,...,0,0,0,1,1,0,0,0,0,0
2917,20,20000,5,7,1960,1996,1224,0,0,1224,...,0,0,0,1,1,0,0,0,0,0
